<figure>
    <img src="../../projects/paintingclf.jpg"
        alt="thumbnail"
        class="post-thumbnail">
    <figcaption>Created by Microsoft Designer</figcaption>
</figure>

In this post, let's build an image classifier to classify paintings made by Picasso and Monet. Let's start by installing the required `python` packages.

## Installing packages

---
code-annotations: hover
---

```bash
!pip install -Uqq fastai # <1>
!pip install -Uqq duckduckgo_search # <2>
```

1. Installs `fastai` library to build and train deep learning models. It also provides utility methods to download images.
2. Installs `duckduckgo_search` library to search for images programatically.

## Import packages

In [2]:
from duckduckgo_search import DDGS
# from fastcore.all import *
from fastdownload import download_url
from time import sleep
from fastai.vision.all import *

ModuleNotFoundError: No module named 'duckduckgo_search'